# COVID-19: United States Confirmed Cases & Deaths 

The following data cleaning and merging is performed to load the data into Power BI for further visualization and analysis.

In [1]:
# Packages / libraries
import os
import numpy as np
import pandas as pd
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

The data collected for this analysis is operated by the Johns Hopkins University Center for Systems Science and Engineering (https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)

In [2]:
# load raw data from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
confirmed = pd.read_csv("C:/Users/garcr/Desktop/time_series_covid19_confirmed_US.csv")
deaths = pd.read_csv("C:/Users/garcr/Desktop/time_series_covid19_deaths_US.csv")

# Confirm that the dataframes have the same number of rows (countries) and columns (dates)
# Deaths dataframe has an extra column 'Population'
print('The shape of confirmed is:', confirmed.shape)
print('The shape of deaths is:', deaths.shape)

confirmed.head()
#deaths.head()

The shape of confirmed is: (3340, 345)
The shape of deaths is: (3340, 346)


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3233,3258,3300,3329,3426,3510,3570,3647,3698,3741
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,10489,10665,10806,10898,11061,11212,11364,11556,11722,11827
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1264,1269,1272,1275,1292,1296,1309,1318,1330,1336
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,1398,1417,1441,1455,1504,1520,1548,1577,1601,1613
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,3663,3744,3776,3803,3881,3950,4036,4118,4191,4218


In [3]:
# Drop unnecessary columns
confirmed.drop(['iso2','iso3','code3','FIPS','Combined_Key'], axis=1, inplace=True)
deaths.drop(['iso2','iso3','code3','FIPS','Combined_Key'], axis=1, inplace=True)

#confirmed.head()
#deaths.head()

In [4]:
# melt the date columns into one column
confirmed2 = pd.melt(confirmed, id_vars=[
                     'UID', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_'], var_name=['Date'])
deaths2 = pd.melt(deaths, id_vars=['UID', 'Admin2', 'Province_State',
                                   'Country_Region', 'Lat', 'Long_', 'Population'], var_name=['Date'])

print('The shape of confirmed is:', confirmed2.shape)
print('The shape of deaths is:', deaths2.shape)

# confirmed2.head()

The shape of confirmed is: (1115560, 8)
The shape of deaths is: (1115560, 9)


In [5]:
# convert Date column into datetime objects (xxxx-xx-xx)
confirmed2['Date'] = pd.to_datetime(confirmed2['Date'])
deaths2['Date'] = pd.to_datetime(deaths2['Date'])

In [6]:
# Replace column headings with detailed fields
confirmed2.columns = confirmed2.columns.str.replace('value', 'Confirmed')
confirmed2.columns = confirmed2.columns.str.replace('Admin2', 'County')
deaths2.columns = deaths2.columns.str.replace('value', 'Deaths')
deaths2.columns = deaths2.columns.str.replace('Admin2', 'County')

# Investigate NULLs before join
print(confirmed2.isnull().sum())
print(deaths2.isnull().sum())

UID                  0
County            2004
Province_State       0
Country_Region       0
Lat                  0
Long_                0
Date                 0
Confirmed            0
dtype: int64
UID                  0
County            2004
Province_State       0
Country_Region       0
Lat                  0
Long_                0
Population           0
Date                 0
Deaths               0
dtype: int64


The null county values are a result of the District of Columbia, and the 5 inhabited US territories (American Samoa, Guam, Northern Mariana Islands, Puerto Rico, US Virgin Islands) not having counties, in addition to the Grand Princess and Diamond Princess cruise ships. The null county value will be filled with the 'Province_State' name of the corresponding territory or cruise ship. 

In [7]:
# Join the dataframes
covid = confirmed2.merge(deaths2[['UID', 'Province_State', 'Country_Region', 'Date', 'Deaths', 'Population']],
                         how='outer',
                         left_on=['UID', 'Province_State',
                                  'Country_Region', 'Date'],
                         right_on=['UID', 'Province_State', 'Country_Region', 'Date'])

# Investigate the shape of the dataframe after the join
print('\nThe shape of confirmed is:', confirmed2.shape)
print('The shape of deaths is:', deaths2.shape)
print('The shape of the joined dataframe is:', covid.shape)

# print(covid.isnull().sum())
# covid.tail()


The shape of confirmed is: (1115560, 8)
The shape of deaths is: (1115560, 9)
The shape of the joined dataframe is: (1115560, 10)


The joined dataframe "covid" has 10 columns because Deaths and Population has been added to the confirmed dataframe. 

In [8]:
# Fill County NaN values with Province_State values (Applies only to DC, US territories, and cruise ships)
covid['County'].fillna(covid['Province_State'], inplace=True)

# covid.head()
covid.isnull().sum()

UID               0
County            0
Province_State    0
Country_Region    0
Lat               0
Long_             0
Date              0
Confirmed         0
Deaths            0
Population        0
dtype: int64

In [9]:
# Add Month-Year column
covid['Month-Year'] = covid['Date'].dt.strftime('%b-%Y')
# covid.tail()

In [10]:
# Copy df
temp = covid.copy()

# Create columns for previous date's COVID-19 cases to create daily numbers and a running total
temp['Temp Date'] = covid['Date'] + pd.Timedelta(days=1)
temp.rename(columns={'Confirmed': 'Confirmed - 1',
                     'Deaths': 'Deaths - 1', 'Date': 'Date - 1'}, inplace=True)

# Perform left join on DFs
covid_19 = covid.merge(temp[['UID', 'Province_State', 'Country_Region', 'Confirmed - 1', 'Deaths - 1',
                             'Temp Date', 'Date - 1']], how='left',
                       left_on=['UID', 'Province_State',
                                'Country_Region', 'Date'],
                       right_on=['UID', 'Province_State', 'Country_Region', 'Temp Date'])
print(covid_19.shape)
# covid_19.head()

(1115560, 15)


In [11]:
# Calculate the daily numbers for confirmed cases & deaths (current aggregate - previous aggregate = daily confirmed)
covid_19['Daily Confirmed'] = covid_19['Confirmed'] - covid_19['Confirmed - 1']
covid_19['Daily Deaths'] = covid_19['Deaths'] - covid_19['Deaths - 1']

print(covid_19.shape)
# covid_19.head()

(1115560, 17)


In [12]:
# Include daily numbers for the first day of data where there is no previous date available (2020-01-22)
covid_19['Daily Confirmed'].loc[covid_19['Date'] == '2020-01-22'] = covid_19['Confirmed']
covid_19['Daily Deaths'].loc[covid_19['Date'] == '2020-01-22'] = covid_19['Deaths']

# Delete unnecessary columns
del covid_19['Confirmed - 1'] 
del covid_19['Deaths - 1']
del covid_19['Temp Date']
del covid_19['Date - 1']

covid_19.head()

,UID,County,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Deaths,Population,Month-Year,Daily Confirmed,Daily Deaths
0,84001001,Autauga,Alabama,US,32.539527,-86.644082,2020-01-22,0,0,55869,Jan-2020,0.0,0.0
1,84001003,Baldwin,Alabama,US,30.727750,-87.722071,2020-01-22,0,0,223234,Jan-2020,0.0,0.0
2,84001005,Barbour,Alabama,US,31.868263,-85.387129,2020-01-22,0,0,24686,Jan-2020,0.0,0.0
3,84001007,Bibb,Alabama,US,32.996421,-87.125115,2020-01-22,0,0,22394,Jan-2020,0.0,0.0
4,84001009,Blount,Alabama,US,33.982109,-86.567906,2020-01-22,0,0,57826,Jan-2020,0.0,0.0


In [13]:
# calculate weekly sum by state (Monday-Sunday)
weekly_covid = covid_19.groupby(['Province_State', covid_19['Date'].dt.strftime(
    '%W')]).agg({'Daily Confirmed': 'sum', 'Daily Deaths': 'sum'}).reset_index()

# replace 'Date' with 'Week Number' of the year
weekly_covid.columns = weekly_covid.columns.str.replace('Date', 'Week Number')
weekly_covid.columns = weekly_covid.columns.str.replace(
    'Daily Confirmed', 'Weekly Confirmed')
weekly_covid.columns = weekly_covid.columns.str.replace(
    'Daily Deaths', 'Weekly Deaths')

# view weekly numbers by state
weekly_covid[weekly_covid['Week Number'] == weekly_covid['Week Number'].max()].head()

,Province_State,Week Number,Weekly Confirmed,Weekly Deaths
47,Alabama,50,26821.0,287.0
95,Alaska,50,2728.0,8.0
143,American Samoa,50,0.0,0.0
191,Arizona,50,45155.0,614.0
239,Arkansas,50,15948.0,292.0


In [14]:
# calculate the weekly percentage change in confirmed cases and deaths for each state
weekly_change = weekly_covid.groupby(['Province_State', 'Week Number']).agg(
    {'Weekly Confirmed': 'sum', 'Weekly Deaths': 'sum'}).pct_change().reset_index()
weekly_change

,Province_State,Week Number,Weekly Confirmed,Weekly Deaths
0,Alabama,03,NaN,NaN
1,Alabama,04,NaN,NaN
2,Alabama,05,NaN,NaN
3,Alabama,06,NaN,NaN
4,Alabama,07,NaN,NaN
...,...,...,...,...
2779,Wyoming,46,0.094715,0.066667
2780,Wyoming,47,-0.238767,0.218750
2781,Wyoming,48,-0.113889,0.307692
2782,Wyoming,49,-0.205068,0.078431


In [15]:
# add the weekly percentage change calculated above to the weekly_covid dataframe
weekly_covid[['Weekly Confirmed % Change', 'Weekly Deaths % Change']
             ] = weekly_change[['Weekly Confirmed', 'Weekly Deaths']]

# replace NaN with 0
weekly_covid.replace(np.nan, 0, inplace=True)

weekly_covid

,Province_State,Week Number,Weekly Confirmed,Weekly Deaths,Weekly Confirmed % Change,Weekly Deaths % Change
0,Alabama,03,0.0,0.0,0.000000,0.000000
1,Alabama,04,0.0,0.0,0.000000,0.000000
2,Alabama,05,0.0,0.0,0.000000,0.000000
3,Alabama,06,0.0,0.0,0.000000,0.000000
4,Alabama,07,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...
2779,Wyoming,46,5675.0,32.0,0.094715,0.066667
2780,Wyoming,47,4320.0,39.0,-0.238767,0.218750
2781,Wyoming,48,3828.0,51.0,-0.113889,0.307692
2782,Wyoming,49,3043.0,55.0,-0.205068,0.078431


In [16]:
# check the most recent daily confirmed cases
daily_sum = covid_19.groupby(['Date']).agg({'Daily Confirmed': 'sum'})
print(daily_sum.tail(10))

# check the most recent daily deaths
daily_deaths = covid_19.groupby(['Date']).agg({'Daily Deaths': 'sum'})
print(daily_deaths.tail(10))

            Daily Confirmed
Date                       
2020-12-11         234103.0
2020-12-12         219193.0
2020-12-13         191142.0
2020-12-14         192846.0
2020-12-15         198766.0
2020-12-16         247403.0
2020-12-17         236211.0
2020-12-18         249709.0
2020-12-19         196295.0
2020-12-20         189099.0
            Daily Deaths
Date                    
2020-12-11        3283.0
2020-12-12        2354.0
2020-12-13        1389.0
2020-12-14        1484.0
2020-12-15        2984.0
2020-12-16        3668.0
2020-12-17        3345.0
2020-12-18        2814.0
2020-12-19        2571.0
2020-12-20        1509.0


In [17]:
# Export Data as csv to load into Power BI
covid_19.to_csv('UScovid', sep='\t')
weekly_covid.to_csv('weekly_covid', sep='\t')